In [11]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv
import pandas as pd

In [123]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


# TABLE DEFINITION

In [154]:
ContactCorrelationTest()

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b id of the postsynaptic neuron,binary_conversion_pearson,binary_conversion_cosine,conversion_pearson,conversion_cosine,density_pearson,density_cosine,syn_volume_mean_pearson,syn_volume_mean_cosine,syn_density_pearson,syn_density_cosine,binary_conversion_pearson_converted,binary_conversion_cosine_converted,conversion_pearson_converted,conversion_cosine_converted,density_pearson_converted,density_cosine_converted,syn_volume_mean_pearson_converted,syn_volume_mean_cosine_converted,syn_density_pearson_converted,syn_density_cosine_converted


In [190]:
ContactCorrelationTest.drop()

`microns_ta3p100`.`contact_correlation_test` (51 tuples)


Proceed? [yes, No]:  yes


Tables dropped.  Restart kernel.


In [191]:
labels_to_keep = [2,3,4,5,6,7,8]

@schema
class ContactCorrelationTest(dj.Manual):
    definition="""
    -> ta3p100.Segment
    segment_b :bigint unsigned #id of the postsynaptic neuron
    ---
    n_seg_a =              :bigint
    n_seg_b =              :bigint
    n_seg_shared           :bigint
    n_seg_shared_converted :bigint 
    binary_conversion_pearson=null :float
    binary_conversion_cosine=null :float
    conversion_pearson=null :float
    conversion_cosine=null :float
    density_pearson=null :float
    density_cosine=null :float
    syn_volume_mean_pearson=null :float
    syn_volume_mean_cosine=null :float
    syn_density_pearson=null :float
    syn_density_cosine=null :float
    #ones that are set by 2nd group
    binary_conversion_pearson_converted=null :float
    binary_conversion_cosine_converted=null :float
    conversion_pearson_converted=null :float
    conversion_cosine_converted=null :float
    density_pearson_converted=null :float
    density_cosine_converted=null :float
    syn_volume_mean_pearson_converted=null :float
    syn_volume_mean_cosine_converted=null :float
    syn_density_pearson_converted=null :float
    syn_density_cosine_converted=null :float



    """
    
    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    
#     def make(self,key):
#         targets = (dj.U("postsyn") & ta3p100.ContactTestSharedContact).proj(segment_id="postsyn") - ta3p100.SegmentExclude
#         info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
#         segment_pairs = info.fetch()
        
#         for (sp in )



#     conversion_corr=null :float #Pearson correlation for n_synapse/n_contact rate for two postsyn segments
#     binary_conversion_simr=null :float #cosine_similarity [0,1] for binary n_synapse/n_contact rate for two postsyn segments
#     density_correlation=null :float #Pearson correlation for n_synapse/postsyn_length rate for two postsyn segments
#     synaptic_size_correlation=null : float #Pearson correlation for mean synaptic volume for two postsyn segments


In [126]:
prepost_data = ta3p100.ContactPrePost2.proj("postsyn","total_contact_conversion",
                "total_contact_density","total_synapse_sizes_mean",
                syn_density="(total_n_synapses*total_synapse_sizes_mean)/total_postsyn_length",
                presyn="segment_id").fetch()






In [131]:
df = pd.DataFrame(prepost_data)
#df[df["presyn"] < 2]["presyn"].to_numpy().any()


In [17]:
targets = (dj.U("postsyn") & ta3p100.Contact).proj(segment_id="postsyn") - ta3p100.SegmentExclude
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
segment_pairs = info.fetch()

In [18]:
len(segment_pairs)

342378

In [113]:
a = np.array([1,1,1,1])
b = np.array([0,0,0,0])

b = np.array([1,1,1,1])
a = np.array([0,0,0,0])

b = np.array([1,0,1,0])
a = np.array([0,1,0,1])

print((b - a))
abs(sum(b - a)) == a.size 

[ 1 -1  1 -1]


False

In [177]:
#calculates the pearson correlation while accounting for the corner cases
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import warnings

def find_pearson(v1,v2):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        if np.array_equal(v1,v2):
            return 1
        elif abs(sum(v1 - v2)) == v1.size:
            return -1
        else:
            #perform the pearson correlation
            corr_conversion, p_value_conversion = pearsonr(v1, v2)
            return corr_conversion

def find_cosine(v1,v2):
    if np.array_equal(v1,v2):
        return 1
    elif abs(sum(v1 - v2)) == v1.size:
        return 0
    else:
        v1 = v1.reshape(1,len(v1))
        v2 = v2.reshape(1,len(v2))
        return cosine_similarity(v1, v2)[0][0]

    

In [145]:
b = np.array([0,0,1,1])
a = np.array([0,0,1,1])
find_cosine(a,b)

0.9999999999999998

In [122]:
b = np.array([0.5]*4)
a = np.array([0.5]*4)
print((find_pearson(a,b)))
np.isnan(find_pearson(a,b))

1


False

In [192]:
#get all of the possible 

from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

total_correlations = []
counter = 0
for postsyn1,postsyn2 in segment_pairs:
    
    start_time = time.time()
    
    
    
    print("postsyn1 = " + str(postsyn1))
    print("postsyn2 = " + str(postsyn2))
    
    #get all of the rows with postsyn 1 and 2
    df_1 = df[df["postsyn"]==postsyn1]
    df_2 = df[df["postsyn"]==postsyn2]
    
    #find the number of presyns for each
    
    #how you restrict a table by the values in the other table
    df_1_common = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])
    df_2_common = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])
    
    
    ###########------------------------------------------------------###########
    #need to get the common axons that have at least one converted contact on one of the postsyns
    """pseudocode
    #get the conversion rates for both tables
    #add them up
    #get the indices that are greater than 0
    #get the presyn ids that match those rows
    #further restrict both groups by those ids
    """
    
    #get both of their conversion rates
    test_1_conv = df_1_common["total_contact_conversion"].to_numpy()
    test_2_conv = df_2_common["total_contact_conversion"].to_numpy()

    test_1_presyn = df_1_common["presyn"].to_numpy()
    new_presyns = test_1_presyn[(test_1_conv + test_2_conv) > 0]

    df_1_common_converted = df_1_common[df_1_common["presyn"].isin(new_presyns)]
    df_2_common_converted = df_2_common[df_2_common["presyn"].isin(new_presyns)]
    ###########------------------------------------------------------###########
    
    dict_segmenation=2
    dict_segment_id=postsyn1
    dict_segment_b=postsyn2

    
    #finds the number of segments, shared_segments and union segments
    n_seg_a = df_1.shape[0]
    n_seg_b = df_2.shape[0]
    n_seg_shared = df_1_common.shape[0]
    n_seg_shared_converted = df_1_common_converted.shape[0]
    
    #initialize the dictionary that will be saved:
    corr_dict = dict(segmentation=2,segment_id=postsyn1,
                                  segment_b=postsyn2,
                                  n_seg_a=n_seg_a,
                                    n_seg_b=n_seg_b,
                                    n_seg_shared=n_seg_shared,
                                    n_seg_shared_converted=n_seg_shared_converted)
    
    
    #initialize the variables that need to be set in the dictionary
    
    
    #ones that are set by 1st group
    binary_conversion_pearson = np.NaN
    binary_conversion_cosine = np.NaN
    conversion_pearson = np.NaN
    conversion_cosine = np.NaN
    density_pearson = np.NaN
    density_cosine = np.NaN
    syn_volume_mean_pearson = np.NaN
    syn_volume_mean_cosine = np.NaN
    syn_density_pearson = np.NaN
    syn_density_cosine = np.NaN
    
    #ones that are set by 2nd group
    binary_conversion_pearson_converted = np.NaN
    binary_conversion_cosine_converted = np.NaN
    conversion_pearson_converted = np.NaN
    conversion_cosine_converted = np.NaN
    density_pearson_converted = np.NaN
    density_cosine_converted = np.NaN
    syn_volume_mean_pearson_converted = np.NaN
    syn_volume_mean_cosine_converted = np.NaN
    syn_density_pearson_converted = np.NaN
    syn_density_cosine_converted = np.NaN

    
    

    if (not df_1_common.to_numpy().any()) or (not df_2_common.to_numpy().any()):
        #write empty dictionary to the database
        #total_correlations.append(corr_dict)
        pass
        
    else:
        #how to extract the data
        #postsyns = restr_df["presyn"].to_numpy()
        # df_1_common_conversion = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
        # df_2_common_conversion = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
        df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
        df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

        df_1_common_binary_conversion = np.copy(df_1_common_conversion)
        df_2_common_binary_conversion = np.copy(df_2_common_conversion)


        df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
        df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

        # print(df_1_common_binary_conversion)
        # print(df_2_common_binary_conversion)

        #postsyns = restr_df["presyn"].to_numpy()
        #df_1_common_density = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
        #df_2_common_density = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
        df_1_common_density = df_1_common["total_contact_density"].to_numpy()
        df_2_common_density = df_2_common["total_contact_density"].to_numpy()



        #get the synaptic volume density
        # df_1_common_synaptic_size = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
        # df_2_common_synaptic_size = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
        df_1_common_synaptic_size = df_1_common["total_synapse_sizes_mean"].to_numpy()
        df_2_common_synaptic_size = df_2_common["total_synapse_sizes_mean"].to_numpy()
        
        df_1_common_syn_density = df_1_common["syn_density"].to_numpy()
        df_2_common_syn_density = df_2_common["syn_density"].to_numpy()

        #calculate the coefficient
        #start_time = time.time()
        
        """
        #ones that are set by 1st group
        binary_conversion_pearson = np.NaN
        binary_conversion_cosine = np.NaN
        conversion_pearson = np.NaN
        conversion_cosine = np.NaN
        density_pearson = np.NaN
        density_cosine = np.NaN
        syn_volume_mean_pearson = np.NaN
        syn_volume_mean_cosine = np.NaN
        syn_volume_per_length_pearson = np.NaN
        syn_volume_per_length_cosine = np.NaN
        
        """
        
        binary_conversion_pearson = find_pearson(df_1_common_binary_conversion, df_2_common_binary_conversion)
        binary_conversion_cosine = find_cosine(df_1_common_binary_conversion, df_2_common_binary_conversion)
        conversion_pearson = find_pearson(df_1_common_conversion, df_2_common_conversion)
        conversion_cosine = find_cosine(df_1_common_conversion, df_2_common_conversion)
        density_pearson = find_pearson(df_1_common_density, df_2_common_density)
        density_cosine = find_cosine(df_1_common_density, df_2_common_density)
        syn_volume_mean_pearson = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
        syn_volume_mean_cosine = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)
        syn_density_pearson = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
        syn_density_cosine = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)
        

        
        
        ####reset the df_1_common and df_1_common to reuse code
        df_1_common = df_1_common_converted
        df_2_common = df_2_common_converted
        
        if (not df_1_common.to_numpy().any()) or (not df_2_common.to_numpy().any()):
            print("none_in_converted")
            pass
        else:
            #how to extract the data
            #postsyns = restr_df["presyn"].to_numpy()
            # df_1_common_conversion = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
            # df_2_common_conversion = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
            df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
            df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

            df_1_common_binary_conversion = np.copy(df_1_common_conversion)
            df_2_common_binary_conversion = np.copy(df_2_common_conversion)


            df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
            df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

            # print(df_1_common_binary_conversion)
            # print(df_2_common_binary_conversion)

            #postsyns = restr_df["presyn"].to_numpy()
            #df_1_common_density = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
            #df_2_common_density = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
            df_1_common_density = df_1_common["total_contact_density"].to_numpy()
            df_2_common_density = df_2_common["total_contact_density"].to_numpy()



            #get the synaptic volume density
            # df_1_common_synaptic_size = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
            # df_2_common_synaptic_size = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
            df_1_common_synaptic_size = df_1_common["total_synapse_sizes_mean"].to_numpy()
            df_2_common_synaptic_size = df_2_common["total_synapse_sizes_mean"].to_numpy()

            df_1_common_syn_density = df_1_common["syn_density"].to_numpy()
            df_2_common_syn_density = df_2_common["syn_density"].to_numpy()

            #calculate the coefficient
            #start_time = time.time()

            """
            #ones that are set by 1st group
            binary_conversion_pearson = np.NaN
            binary_conversion_cosine = np.NaN
            conversion_pearson = np.NaN
            conversion_cosine = np.NaN
            density_pearson = np.NaN
            density_cosine = np.NaN
            syn_volume_mean_pearson = np.NaN
            syn_volume_mean_cosine = np.NaN
            syn_volume_per_length_pearson = np.NaN
            syn_volume_per_length_cosine = np.NaN

            """

            binary_conversion_pearson_converted = find_pearson(df_1_common_binary_conversion, df_2_common_binary_conversion)
            binary_conversion_cosine_converted = find_cosine(df_1_common_binary_conversion, df_2_common_binary_conversion)
            conversion_pearson_converted = find_pearson(df_1_common_conversion, df_2_common_conversion)
            conversion_cosine_converted = find_cosine(df_1_common_conversion, df_2_common_conversion)
            density_pearson_converted = find_pearson(df_1_common_density, df_2_common_density)
            density_cosine_converted = find_cosine(df_1_common_density, df_2_common_density)
            syn_volume_mean_pearson_converted = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
            syn_volume_mean_cosine_converted = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)
            syn_density_pearson_converted = find_pearson(df_1_common_synaptic_size, df_2_common_synaptic_size)
            syn_density_cosine_converted = find_cosine(df_1_common_synaptic_size, df_2_common_synaptic_size)


            
            
            
            
            
        
#         corr_conversion, p_value_conversion = pearsonr(df_1_common_conversion, df_2_common_conversion)
#         corr_density, p_value_density = pearsonr(df_1_common_density, df_2_common_density)
#         df_1_common_binary_conversion = df_1_common_binary_conversion.reshape(1,len(df_1_common_binary_conversion))
#         df_2_common_binary_conversion = df_2_common_binary_conversion.reshape(1,len(df_2_common_binary_conversion))
#         cos_binary_conversion = cosine_similarity(df_1_common_binary_conversion, df_2_common_binary_conversion)[0][0]

#         corr_synaptic_size, p_value_synaptic = pearsonr(df_1_common_synaptic_size, df_2_common_synaptic_size)

#         print("corr_conversion = " + str(corr_conversion))
#         print("cos_binary_conversion = " + str(cos_binary_conversion))
#         print("corr_density = " + str(corr_density))
#         print("corr_synaptic_size = " + str(corr_synaptic_size))



    #find the number of contacts and shared contacts
    
    corr_dict["binary_conversion_pearson"] = binary_conversion_pearson
    corr_dict["binary_conversion_cosine"] = binary_conversion_cosine
    corr_dict["conversion_pearson"] = conversion_pearson
    corr_dict["conversion_cosine"] = conversion_cosine
    corr_dict["density_pearson"] = density_pearson
    corr_dict["density_cosine"] = density_cosine
    corr_dict["syn_volume_mean_pearson"] = syn_volume_mean_pearson
    corr_dict["syn_volume_mean_cosine"] = syn_volume_mean_cosine
    corr_dict["syn_density_pearson"] = syn_density_pearson
    corr_dict["syn_density_cosine"] = syn_density_cosine

    corr_dict["binary_conversion_pearson_converted"] = binary_conversion_pearson_converted
    corr_dict["binary_conversion_cosine_converted"] = binary_conversion_cosine_converted
    corr_dict["conversion_pearson_converted"] = conversion_pearson_converted
    corr_dict["conversion_cosine_converted"] = conversion_cosine_converted
    corr_dict["density_pearson_converted"] = density_pearson_converted
    corr_dict["density_cosine_converted"] = density_cosine_converted
    corr_dict["syn_volume_mean_pearson_converted"] = syn_volume_mean_pearson_converted
    corr_dict["syn_volume_mean_cosine_converted"] = syn_volume_mean_cosine_converted
    corr_dict["syn_density_pearson_converted"] = syn_density_pearson_converted
    corr_dict["syn_density_cosine_converted"] = syn_density_cosine_converted



    # print(corr_synaptic_size)
    #print(f"Total time for correlation = {time.time()-start_time}")



    total_correlations.append(corr_dict)
        
#         conversion_corr=null :float #Pearson correlation for n_synapse/n_contact rate for two postsyn segments
#     binary_conversion_corr=null :float #cosine_similarity [0,1] for binary n_synapse/n_contact rate for two postsyn segments
#     density_correlation=null :float #Pearson correlation for n_synapse/postsyn_length rate for two postsyn segments
#     synaptic_size_correlation=null : float #Pearson correlation for mean synaptic volume for two postsyn segments
#     "
    #write the dictionary to th
    #print(corr_dict)
    #total_correlations.append(corr_dict)

    counter += 1
    if counter > 50:
        break

#write all of the dictionaries to the database



postsyn1 = 648518346341353574
postsyn2 = 648518346341357297
none_in_converted
Total time for correlation = 0.036835670471191406
postsyn1 = 648518346341355539
postsyn2 = 648518346341357297
Total time for correlation = 0.03626132011413574
postsyn1 = 648518346341353186
postsyn2 = 648518346341357297
Total time for correlation = 0.030786514282226562
postsyn1 = 648518346341354962
postsyn2 = 648518346341357297
Total time for correlation = 0.024353504180908203
postsyn1 = 648518346341353607
postsyn2 = 648518346341357297
Total time for correlation = 0.02694845199584961
postsyn1 = 648518346341356260
postsyn2 = 648518346341357297
Total time for correlation = 0.02962946891784668
postsyn1 = 648518346341352223
postsyn2 = 648518346341357297
Total time for correlation = 0.04384732246398926
postsyn1 = 648518346341353883
postsyn2 = 648518346341357297
Total time for correlation = 0.022904157638549805
postsyn1 = 648518346341352006
postsyn2 = 648518346341357297
Total time for correlation = 0.027090072631835

In [193]:
#corr_dict["syn_density_cosine_converted"] = np.nan
ContactCorrelationTest.insert(total_correlations,skip_duplicates=True)

In [194]:
ContactCorrelationTest()

segmentation segmentation id,segment_id segment id unique within each Segmentation,segment_b id of the postsynaptic neuron,n_seg_a,n_seg_b,n_seg_shared,n_seg_shared_converted,binary_conversion_pearson,binary_conversion_cosine,conversion_pearson,conversion_cosine,density_pearson,density_cosine,syn_volume_mean_pearson,syn_volume_mean_cosine,syn_density_pearson,syn_density_cosine,binary_conversion_pearson_converted,binary_conversion_cosine_converted,conversion_pearson_converted,conversion_cosine_converted,density_pearson_converted,density_cosine_converted,syn_volume_mean_pearson_converted,syn_volume_mean_cosine_converted,syn_density_pearson_converted,syn_density_cosine_converted
2,648518346341352006,648518346341357297,1420,5077,464,69,-0.0193943,0.0573068,-0.0139197,0.0542704,0.087484,0.126612,0.0961249,0.130221,0.0961249,0.130221,-0.941647,0.0573068,-0.736381,0.0542704,-0.227271,0.126612,-0.166053,0.130221,-0.166053,0.130221
2,648518346341352006,648518346341366977,1420,402,27,4,-0.0693375,0.0,-0.0693375,0.0,-0.0693306,0.0,-0.0469759,0.0,-0.0469759,0.0,-1.0,0.0,-1.0,0.0,-0.999647,0.0,-0.438921,0.0,-0.438921,0.0
2,648518346341352223,648518346341357297,1262,5077,360,48,0.0287361,0.0890871,0.131245,0.169414,-0.0125937,0.0202922,0.00100889,0.0265834,0.00100889,0.0265834,-0.899735,0.0890871,-0.312201,0.169414,-0.346253,0.0202922,-0.206104,0.0265834,-0.206104,0.0265834
2,648518346341352223,648518346341366977,1262,402,11,1,nan,0.0,nan,0.0,nan,0.0,nan,0.0,nan,0.0,-1.0,0.0,-1.0,0.0,nan,0.0,nan,0.0,nan,0.0
2,648518346341353019,648518346341357297,76,5077,3,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341353019,648518346341366977,76,402,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341353186,648518346341357297,1420,5077,574,75,0.0221271,0.0833333,0.0216621,0.0737666,0.0512278,0.0722423,0.00134567,0.0246918,0.00134567,0.0246918,-0.909059,0.0833333,-0.637754,0.0737666,-0.119881,0.0722423,-0.194431,0.0246918,-0.194431,0.0246918
2,648518346341353186,648518346341366977,1420,402,47,4,-0.0384995,0.0,-0.0384995,0.0,-0.0379605,0.0,-0.0258678,0.0,-0.0258678,0.0,-1.0,0.0,-1.0,0.0,-0.950397,0.0,-0.42447,0.0,-0.42447,0.0
2,648518346341353574,648518346341357297,208,5077,11,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,648518346341353574,648518346341366977,208,402,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [137]:
df_1_common

,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density,total_synapse_sizes_mean,syn_density
6070,2,648518346341352285,648518346341353574,0.0,0.0,0.0,0
54723,2,648518346341357484,648518346341353574,0.0,0.0,0.0,0
139446,2,648518346341362945,648518346341353574,0.0,0.0,0.0,0
160843,2,648518346341363964,648518346341353574,0.0,0.0,0.0,0
282057,2,648518346341370685,648518346341353574,0.0,0.0,0.0,0
295452,2,648518346341371435,648518346341353574,0.0,0.0,0.0,0
444085,2,648518346341388248,648518346341353574,0.0,0.0,0.0,0
503702,2,648518346341394182,648518346341353574,0.0,0.0,0.0,0
793003,2,648518346341403948,648518346341353574,0.0,0.0,0.0,0
1306309,2,648518346342792143,648518346341353574,0.0,0.0,0.0,0


In [149]:
df_1_common_converted

,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density,total_synapse_sizes_mean,syn_density


In [186]:
test_1 = [dict(segmentation=2,presyn=648518346341352285,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341357484,total_contact_conversion=0.12),
 dict(segmentation=2,presyn=648518346341362945,total_contact_conversion=0.13),
 dict(segmentation=2,presyn=648518346341363964,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341370685,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341371435,total_contact_conversion=0.5),
 dict(segmentation=2,presyn=648518346341388248,total_contact_conversion=0),
]

test_1_df = pd.DataFrame(test_1)

test_2 = [dict(segmentation=2,presyn=648518346341352285,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341357484,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341362945,total_contact_conversion=0.13),
 dict(segmentation=2,presyn=648518346341363964,total_contact_conversion=0.1),
 dict(segmentation=2,presyn=648518346341370685,total_contact_conversion=0),
 dict(segmentation=2,presyn=648518346341371435,total_contact_conversion=0.5),
 dict(segmentation=2,presyn=648518346341388248,total_contact_conversion=0),
]

test_2_df = pd.DataFrame(test_2)

#get both of their conversion rates
test_1_conv = test_1_df["total_contact_conversion"].to_numpy()
test_2_conv = test_2_df["total_contact_conversion"].to_numpy()

(test_1_conv + test_2_conv) > 0

test_1_presyn = test_1_df["presyn"].to_numpy()
new_presyns = test_1_presyn[(test_1_conv + test_2_conv) > 0]

test_1_df[test_1_df["presyn"].isin(new_presyns)]

,presyn,segmentation,total_contact_conversion
1,648518346341357484,2,0.12
2,648518346341362945,2,0.13
3,648518346341363964,2,0.00
5,648518346341371435,2,0.50


In [189]:
test_1_df[test_1_df["presyn"].isin(new_presyns)].shape[0]

4

In [85]:
df_1_common

,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density,total_synapse_sizes_mean
6070,2,648518346341352285,648518346341353574,0.0,0.0,0.0
54723,2,648518346341357484,648518346341353574,0.0,0.0,0.0
139446,2,648518346341362945,648518346341353574,0.0,0.0,0.0
160843,2,648518346341363964,648518346341353574,0.0,0.0,0.0
282057,2,648518346341370685,648518346341353574,0.0,0.0,0.0
295452,2,648518346341371435,648518346341353574,0.0,0.0,0.0
444085,2,648518346341388248,648518346341353574,0.0,0.0,0.0
503702,2,648518346341394182,648518346341353574,0.0,0.0,0.0
793003,2,648518346341403948,648518346341353574,0.0,0.0,0.0
1306309,2,648518346342792143,648518346341353574,0.0,0.0,0.0


In [184]:
df_2_common

,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density,total_synapse_sizes_mean,syn_density


In [183]:
df_2_common.shape

(0, 7)

In [80]:
df_new = df_1_common.copy(deep=True)
df_new.at['54723', 'total_contact_conversion'] = 0.2
df_new
# df_1_common.iloc[0][3] = 1
# df_1_common.at['54723', 'total_contact_conversion'] = 1


,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density,total_synapse_sizes_mean
6070,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
54723,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
139446,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
160843,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
282057,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
295452,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
444085,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
503702,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
793003,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0
1306309,2.0,6.485183e+17,6.485183e+17,0.0,0.0,0.0


In [81]:
sum(df_new["total_contact_conversion"]>0.0)

1

In [5]:
segment_pairs[1]

(648518346341352223, 648518346341353019)

In [62]:
import pandas as pd

#pull down the PrePost
prepost_data = ta3p100.ContactPrePostTestReal.proj("presyn",
        "postsyn","total_contact_conversion","total_contact_density","total_synapse_sizes_mean").fetch()

# prepost_data = ta3p100.ContactPrePostTestReal.proj("presyn",
#         "postsyn","total_contact_conversion","total_contact_density").fetch()


#put data in Dataframe
df = pd.DataFrame(prepost_data)

start_time = time.time()
#get one of the possible combinations from segment pairs
postsyn1 = 648518346341353019
postsyn2 = 648518346341352223

#get all of the rows with postsyn 1 and 2
df_1 = df[df["postsyn"]==postsyn1][:5]
df_2 = df[df["postsyn"]==postsyn2][1:]
#how you restrict a table by the values in the other table
df_1_common = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])
df_2_common = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])
#how to extract the data
#postsyns = restr_df["presyn"].to_numpy()
# df_1_common_conversion = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
# df_2_common_conversion = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
# df_1_common_conversion = df_1_common["total_contact_conversion"].to_numpy()
# df_2_common_conversion = df_2_common["total_contact_conversion"].to_numpy()

# df_1_common_binary_conversion = np.copy(df_1_common_conversion)
# df_2_common_binary_conversion = np.copy(df_2_common_conversion)


# df_1_common_binary_conversion[df_1_common_binary_conversion>0] = 1.0
# df_2_common_binary_conversion[df_2_common_binary_conversion>0] = 1.0

# # print(df_1_common_binary_conversion)
# # print(df_2_common_binary_conversion)

# #postsyns = restr_df["presyn"].to_numpy()
# #df_1_common_density = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
# #df_2_common_density = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
# df_1_common_density = df_1_common["total_contact_density"].to_numpy()
# df_2_common_density = df_2_common["total_contact_density"].to_numpy()



# #get the synaptic volume density
# # df_1_common_synaptic_size = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
# # df_2_common_synaptic_size = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_synapse_sizes_mean"].to_numpy()
# df_1_common_synaptic_size = df_1_common["total_synapse_sizes_mean"].to_numpy()
# df_2_common_synaptic_size = df_2_common["total_synapse_sizes_mean"].to_numpy()

# #calculate the coefficient
# from scipy.stats import pearsonr
# from sklearn.metrics.pairwise import cosine_similarity
# #start_time = time.time()

# corr_conversion, p_value_conversion = pearsonr(df_1_common_conversion, df_2_common_conversion)
# corr_density, p_value_density = pearsonr(df_1_common_density, df_2_common_density)
# df_1_common_binary_conversion = df_1_common_binary_conversion.reshape(1,len(df_1_common_binary_conversion))
# df_2_common_binary_conversion = df_2_common_binary_conversion.reshape(1,len(df_2_common_binary_conversion))
# cos_binary_conversion = cosine_similarity(df_1_common_binary_conversion, df_2_common_binary_conversion)[0][0]

# corr_synaptic_size, p_value_synaptic = pearsonr(df_1_common_synaptic_size, df_2_common_synaptic_size)

# print("corr_conversion = " + str(corr_conversion))
# print("cos_binary_conversion = " + str(cos_binary_conversion))
# print("corr_density = " + str(corr_conversion))
# print("corr_synaptic_size = " + str(corr_synaptic_size))


# # print(corr_synaptic_size)
# print(f"Total time for correlation = {time.time()-start_time}")




DataJointError: Attribute `total_synapse_sizes_mean` is not found

In [61]:
#testing the correlation:

from scipy.stats import pearsonr
v1 = np.array([0.11]*6)
v2 = np.array([0,1,1,1,0,1])


v1 = np.array([0.1,0,1,0,1])
v2 = np.array([1,0,1,0,1,0])



print("v1 = "+str(v1))
print("v2 = "+str(v2))
    
corr_density, p_value_density = pearsonr(v1,v2)
#print(angle_vectors(v1,v2))
print("pearson correlation = " + str(corr_density))
print("is nan pearson = " + str(np.isnan(corr_density)))

v1 = v1.reshape(1,len(v1))
v2 = v2.reshape(1,len(v2))
cos_binary_conversion = cosine_similarity(df_1_common_binary_conversion, df_2_common_binary_conversion)[0][0]
print("cos_binary_conversion = " + str(cos_binary_conversion))


v1 = [0.11 0.11 0.11 0.11 0.11 0.11]
v2 = [0 1 1 1 0 1]
pearson correlation = nan
is nan pearson = True
cos_binary_conversion = 0.0


In [38]:


def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [41]:
from numpy import (array, dot, arccos, clip)
from numpy.linalg import norm
def angle_vectors(u,v):
    return dot(u,v)/norm(u)/norm(v)